## Create a system mixing SMIRNOFF and non-SMIRNOFF-formatted force fields

### Parametrize a molecule with the Sage force field

First, we parametrize the ligand (toluene) with the SMIRNOFF-format Sage force field through the usual route to create an OpenMM `System`.

In [ ]:
try:
    from openmm import app
    from openmm import unit as openmm_unit
except ImportError:
    from simtk import unit as openmm_unit
    from simtk.openmm import app

import numpy as np
from openff.interchange.components.interchange import Interchange
from openff.units import unit

from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path


def converter(x):
    return x.value_in_unit(openmm_unit.nanometer)

In [ ]:
toluene_positions = app.PDBFile(get_data_file_path("molecules/toluene.pdb")).positions
toluene_topology = Molecule.from_smiles("Cc1ccccc1").to_topology()

sage = ForceField("openff-2.0.0.offxml")

toluene = Interchange.from_smirnoff(force_field=sage, topology=toluene_topology)
toluene.positions = np.frompyfunc(converter, 1, 1)(toluene_positions).astype(float)

In [ ]:
ff14sb = ForceField("ff14sb_off_impropers_0.0.1.offxml")
ff14sb.deregister_parameter_handler("ImproperTorsions")

# TODO: This PDB file also has water an ions, which are not yet intelligently parsed here.
#       Unclear if we should have a Topology.from_pdb, just drop the waters, hack through OpenMM ...
t4_lysozyme_molecule = Molecule.from_pdb(
    get_data_file_path("systems/test_systems/T4_lysozyme_water_ions.pdb")
)

t4_lysozyme = Interchange.from_smirnoff(
    force_field=ff14sb, topology=t4_lysozyme_molecule.to_topoplogy()
)
t4_lysozyme.positions = t4_lysozyme_molecule.conformers[0]

In [ ]:
complex_system = t4_lysozyme + toluene

In [ ]:
complex_system.to_openmm()